In [ ]:
#!/usr/bin/env python3
import rpyc
import time
import cv2
from concurrent.futures import Executor, ThreadPoolExecutor
from threading import Thread

from cv2 import imshow
import numpy as np
import HandTrakingModule as htm
import math
velocidadRotores = 200
notificacion1 = "Advertencia stop"
notificacion2 = "Advertencia peatones cercanos"
notificacion3 = "Advertencia coches cercanos"
senal1 = 0 
senal2 = 0
senal3 = 0
conn = rpyc.classic.connect('192.168.1.80') # host name or IP address of the EV3
ev3 = conn.modules['ev3dev.ev3']      # import ev3dev.ev3 remotely


def siguelineas(): 
    s = ev3.ColorSensor()
    leftMotor = ev3.LargeMotor('outB')
    rightMotor = ev3.LargeMotor('outC')
    global velocidadRotores
    startTime = time.time()
    while time.time() < startTime + 500:
    #while True:  
        if s.reflected_light_intensity > 60:
            velocidad = velocidadRotores + 50
            leftMotor.speed_sp = velocidad
            rightMotor.speed_sp = 100

        else: 
            velocidad = velocidadRotores + 50
            leftMotor.speed_sp = 100
            rightMotor.speed_sp = velocidad
        print(velocidad)
        leftMotor.run_forever()
        rightMotor.run_forever()

    leftMotor.stop()
    rightMotor.stop()
    
def detector_moviles(): 
    
    global senal3

    video=cv2.VideoCapture(1)
    address = "https://192.168.1.33:8080/video"
    video.open(address)
    car_cascade = cv2.CascadeClassifier('cars.xml')
  

    while True:

        ret, frames = video.read()

        gray = cv2.cvtColor(frames, cv2.COLOR_BGR2GRAY)
        cars = car_cascade.detectMultiScale(gray, 1.3, 1)

        for (x,y,w,h) in cars:
            cv2.rectangle(frames,(x,y),(x+w,y+h),(0,0,255),2)
            senal3 = senal3 + 1

        cv2.imshow('video2', frames)
        if cv2.waitKey(33) == 27:
            break

    cv2.destroyAllWindows()


def detectorpeatones_senales(): 

    video=cv2.VideoCapture(1)
    address = "https://192.168.1.33:8080/video"
    video.open(address)
    global senal1
    global senal2

    print("Press any key to exit...") 

    cv2.waitKey(1)

    while(True):
        check, frame = video.read()
        gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)

        #Parte detección de peatones-------------------------------------------------------------------
        
        parte_arriba = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_upperbody.xml')

        peatones = parte_arriba.detectMultiScale(frame, 1.3, 1)

        #----------------------------------------------------------------------------------------------
        #Parte deteción de señales---------------------------------------------------------------------

        stop = cv2.CascadeClassifier('xml/stop.xml')
        senales = stop.detectMultiScale(frame, 1.3, 1)
     
        #----------------------------------------------------------------------------------------------
        
        for (x,y,w,h) in peatones:
            cv2.rectangle(frame,(x,y),(x+w,y+h),(128,0,128),2)
            senal1 = senal1 + 1

        for (x,y,w,h) in senales:
            cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
            senal2 = senal2 + 1
            
        cv2.imshow("PERSONAL", frame)
        key=cv2.waitKey(1)

        if key==ord('q'):
            break
    video.release()
    cv2.destroyAllWindows

def speedcontrol(): 
    wCam, hCam = 480, 360
    global velocidadRotores
    global notificacion1
    global notificacion2
    global notificacion3
    global senal1
    global senal2
    global senal3
    cap = cv2.VideoCapture(0)
    cap.set(3, wCam)
    cap.set(4, hCam)
    pTime = 0

    detector = htm.handDetector(detectionCon=0.7)

    while True: 
        success, img = cap.read()
        img = detector.findHands(img)
        lmList = detector.findPosition(img, draw=False)
        if len(lmList) != 0: 
            #print(lmList[4], lmList[8])


            x1, y1 = lmList[4][1], lmList[4][2]
            x2, y2 = lmList[8][1], lmList[8][2]
            cx, cy = (x1 + x2) // 2, (y1 + y2) // 2

            cv2.circle(img, (x1, y1), 15, (255, 0, 255), cv2.FILLED)
            cv2.circle(img, (x2, y2), 15, (255, 0, 255), cv2.FILLED)
            cv2.line(img, (x1, y1), (x2, y2), (255, 0, 255), 3)
            cv2.circle(img, (cx, cy), 15, (255, 0, 255), cv2.FILLED)

            velocidadRotores = math.hypot(x2 - x1, y2 - y1)
            #length = math.hypot(x2 - x1, y2 - y1)
            #print(length)

            if velocidadRotores<50: 
                cv2.circle(img, (cx, cy), 15, (0, 255, 0), cv2.FILLED)
            

        cTime = time.time()
        fps = 1/(cTime-pTime)
        pTime = cTime
        cv2.putText(img, f'FPS: {int(fps)}', (40, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0, 0), 3)
        cv2.putText(img, notificacion2, (0, 290), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 0, 255), 3)
        cv2.putText(img, notificacion1, (0, 350), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 0, 255), 3)
        cv2.putText(img, notificacion3, (0, 230), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 0, 255), 3)
        if senal1 > 3: 
            cv2.putText(img, notificacion2, (0, 290), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 0), 3)
            senal1 = 0
        if senal2 > 3: 
            cv2.putText(img, notificacion1, (0, 350), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 0), 3)
            senal2 = 0
        if senal3 > 3: 
            cv2.putText(img, notificacion3, (0, 230), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 0), 3)
            senal3 = 0
        
        cv2,imshow("Img", img)
        cv2.waitKey(1)


thread1 = Thread(target=speedcontrol)
thread2 = Thread(target=detectorpeatones_senales)
thread3 = Thread(target=siguelineas)
thread4 = Thread(target=detector_moviles)

thread1.start()
thread2.start()
thread3.start()
thread4.start()

thread1.join()
thread2.join()
thread3.join()
thread4.join()



Press any key to exit...
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
58.60232526704263
83.94112549695429
142.7793080379456
142.7793080379456
142.7793080379456
142.7793080379456
142.7793080379456
142.7793080379456
142.7793080379456
142.7793080379456
142.7793080379456
144.0
164.3547113152755
176.5701386583739
188.31124321616085
191.4213562373095
197.50254235097103
197.57371039585604
164.00438588054408
144.19129471453294
145.0841732361385
153.04368005850722
133.2165848854662
57.28010988928052
57.071067811865476
175.03